# <span style='color:blue'> ***NAIVE BAYES***

This notebook explores a Naive Bayes approach to toxicity classification. The goal here is to identify toxic content in text.  
We'll preprocess the data and evaluate the performance of Naive Bayes as a baseline classifier for this task.

In [1]:
import pandas as pd
import numpy as np
import string
import spacy
import re
import html
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('toxicity/train.csv',usecols=['comment_text','malignant'])

Creating a dataset with balanced classes

In [3]:
positive_samples = df[df['malignant']==1]
negative_samples = df[df['malignant']==0].sample(positive_samples['malignant'].count())

In [4]:
data = pd.concat([positive_samples,negative_samples])

Splitting data into training and testing set

In [5]:
train_data, test_data = train_test_split(data,test_size=0.2)

In [6]:
X_train, y_train = np.array(train_data['comment_text']), np.array(train_data['malignant'])
X_test, y_test = np.array(test_data['comment_text']), np.array(test_data['malignant'])

Preprocessing data by removing stopwords, links, html tags and punctuation

In [7]:
trash_tokens = list(STOP_WORDS) + list(string.punctuation) + list(" ")
nlp = spacy.load('en_core_web_sm')

In [8]:
def preprocess_data(X):
    if isinstance(X,str):
        text = re.sub(r'<.*?>', '', X)
        text = re.sub(r'http\S+|www\S+', '', text)
        text = html.unescape(text)
        text = re.sub(r'\n|\t|\r', ' ', text)
        text = text.strip()
        tokens = nlp(text)
        return [str(word).lower() for word in tokens if str(word).lower() not in trash_tokens]
    
    tmp = []
    for i in range(X.shape[0]):
        text = re.sub(r'<.*?>', '', X[i])
        text = re.sub(r'http\S+|www\S+', '', text)
        text = html.unescape(text)
        text = re.sub(r'\n|\t|\r', ' ', text)
        text = text.strip()
        tokens = nlp(text)
        tmp.append([str(word).lower() for word in tokens if str(word).lower() not in trash_tokens])
    
    return tmp

In [9]:
filtered_X_train = preprocess_data(X_train)

In [10]:
def get_word_frequency(X,Y):
    word_dict = {}

    for i in range(len(X)):
        text = X[i]
        label = Y[i]

        for word in text:
            if word not in word_dict:
                word_dict[word] = {'toxic':1,'normal':1}

            if label == 1:
                word_dict[word]['toxic'] += 1
            else:
                word_dict[word]['normal'] += 1

    return word_dict

In [11]:
word_freq = get_word_frequency(filtered_X_train, y_train)
class_frequency = {'toxic':sum(y_train==1), 'normal':sum(y_train==0)}

In [12]:
def prob_word_given_class(word,label,word_freq,class_freq):
    prob_word = word_freq[word][label] / class_freq[label]
    
    return prob_word

In [13]:
def prob_sentence_given_class(sentence,label,word_freq,class_freq):
    prob_sentence = 1
    for word in sentence:
        if word in word_freq.keys():
            prob_sentence *= prob_word_given_class(word,label,word_freq,class_freq)
    
    return prob_sentence

In [14]:
def naive_bayes_classifier(X,word_freq,class_freq,return_likelihoods=False):
    prob_toxic = class_frequency['toxic'] / (class_frequency['toxic'] + class_frequency['normal'])
    prob_normal = class_frequency['normal'] / (class_frequency['toxic'] + class_frequency['normal'])
    prob_sentence_given_toxic = prob_sentence_given_class(X,'toxic',word_freq,class_freq)
    prob_sentence_given_normal = prob_sentence_given_class(X,'normal',word_freq,class_freq)

    toxic_likelihood = prob_toxic * prob_sentence_given_toxic
    normal_likelihood = prob_normal * prob_sentence_given_normal

    if return_likelihoods:
        return(toxic_likelihood, normal_likelihood)

    if toxic_likelihood >= normal_likelihood:
        return 1
    elif normal_likelihood > toxic_likelihood:
        return 0

In [15]:
y_pred = []
for i in range(len(X_test)):
    y_pred.append(naive_bayes_classifier(preprocess_data(X_test[i]),word_freq,class_frequency))

In [32]:
def compute_metrics(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    true_positive = sum((y_true & y_pred) == 1)
    true_negative = sum((y_true & y_pred) == 0)
    false_negative = sum((y_true == 1) & (y_pred == 0))
    false_positive = sum((y_true == 0) & (y_pred == 1))

    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / (true_positive + false_negative)
    accuracy = (true_positive + true_negative) / (true_positive + true_negative + false_positive + false_negative)
    f1 = 2 * (precision * recall) / (precision + recall)

    return {
        'accuracy' : accuracy,
        'precision' : precision,
        'recall' : recall,
        'f1' : f1
    } 

In [33]:
compute_metrics(y_test, y_pred)

{'accuracy': 0.8641242937853107,
 'precision': 0.9063214013709063,
 'recall': 0.7687338501291989,
 'f1': 0.8318769660957708}

In [55]:
def log_prob_sentence_given_class(sentence,label,word_freq,class_freq):
    log_prob_sentence = 0
    for word in sentence:
        if word in word_freq.keys():
            log_prob_sentence += np.log(prob_word_given_class(word,label,word_freq,class_freq))
    
    return log_prob_sentence

In [60]:
def log_naive_bayes_classifier(X,word_freq,class_freq,return_likelihoods=False):
    prob_toxic = class_frequency['toxic'] / (class_frequency['toxic'] + class_frequency['normal'])
    prob_normal = class_frequency['normal'] / (class_frequency['toxic'] + class_frequency['normal'])
    log_prob_sentence_given_toxic = log_prob_sentence_given_class(X,'toxic',word_freq,class_freq)
    log_prob_sentence_given_normal = log_prob_sentence_given_class(X,'normal',word_freq,class_freq)

    log_toxic_likelihood = np.log(prob_toxic) + log_prob_sentence_given_toxic
    log_normal_likelihood = np.log(prob_normal) + log_prob_sentence_given_normal

    if return_likelihoods:
        return(log_toxic_likelihood, log_normal_likelihood)

    if log_toxic_likelihood >= log_normal_likelihood:
        return 1
    elif log_normal_likelihood > log_toxic_likelihood:
        return 0

In [61]:
y_pred_log = []
for i in range(len(X_test)):
    y_pred_log.append(log_naive_bayes_classifier(preprocess_data(X_test[i]),word_freq,class_frequency))

In [62]:
compute_metrics(y_test, y_pred_log)

{'accuracy': 0.8706418101608083,
 'precision': 0.9313609467455621,
 'recall': 0.7625968992248062,
 'f1': 0.838572189664358}